## Contributors
### Jay Upadhyay
### 
###
###
###

# Libraries

In [ ]:
# from tensorflow import keras
# from keras import utils
# from keras.models import Sequential
# from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import numpy as np
import h5py
import pandas as pd
import seaborn as sns

In [ ]:
df_orig=pd.read_csv("adult.csv")
df_orig.dropna(inplace=True)
df_orig

# 1. Project Proposal


We pulled the data from https://archive-beta.ics.uci.edu/dataset/2/adult .

# 2. Data Collection and data cleaning


## Changing workclass to numeric

In [ ]:
df_orig[" workclass"].unique()

In [ ]:
df_orig[" workclass"].replace([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'],
                        range(0,9), inplace=True)

## Changing Education to Numeric

In [ ]:
df_orig.education.unique()


#### Note. I want to come back to this and bin it into appropriate 
I also chose to exclude this for now because I believe education-num already represents it numerically.

In [ ]:
# df_orig['education'].replace([ ' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad',
#                               ' Assoc-voc',' Assoc-acdm','Some-college',' Bachelors', ' Masters',
#                               ' Prof-school' ,' Doctorate'],
#                         [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], inplace=True)


## Changing marital status to numeric

In [ ]:
df_orig["marital-status"].unique()

In [ ]:
df_orig["marital-status"].replace([' Never-married', ' Married-civ-spouse', ' Divorced',
       ' Married-spouse-absent', ' Separated', ' Married-AF-spouse',
       ' Widowed'],range(1,8), inplace=True)
df_orig



## Occupation

I don't like the ? mark section.

In [ ]:
df_orig["occupation"].unique()

In [ ]:
df_orig["occupation"].replace([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', ' ?', ' Protective-serv', ' Armed-Forces',
       ' Priv-house-serv'],range(0,15), inplace=True)
df_orig

## Relationship conversion to numeric

I don't like other relative section here.


In [ ]:
df_orig["relationship"].unique()

In [ ]:
df_orig["relationship"].replace([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried',
       ' Other-relative'],range(0,6), inplace=True)
df_orig

## Race conversion to numeric


In [ ]:
df_orig["race"].unique()

In [ ]:
df_orig["race"].replace([' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other'],range(0,5), inplace=True)
df_orig

## Sex conversion to numeric

In [ ]:
df_orig["sex"].unique()

In [ ]:
df_orig["sex"].replace([' Male', ' Female'],range(0,2), inplace=True)
df_orig

#Country to Numeric

In [ ]:
# Changing a column "class" to be 0:<=50K and 1:>50K
country_list = list(sorted(set(df_orig["native-country"])))

for i,x in enumerate(country_list):
    print(i,":", x)

df_orig['native-country'].replace(country_list,
                        range(len(country_list)), inplace=True)

# Income to numeric

In [ ]:
# Changing a column "class" to be 0:<=50K and 1:>50K
class_list = list(sorted(set(df_orig["class"])))

for i,x in enumerate(class_list):
    print(i,":", x)
    
df_orig['class'].replace(class_list,
                        range(len(class_list)), inplace=True)

In [ ]:
df_orig

## Correlation Matrix

In [ ]:

df_small = df_orig.iloc[:,[0,1,3,8,9,12,13,14]]
correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()

# 3. EDA

# 4. Techniques Used

# 5. Project Proposal